In [1]:
#-----------------------------------------------------------------------------------------------------
# 문서들을 embed하는 예제-2
# - 여기서는 문서들을 전처리하고, 임베딩 하는 과정임.
#
# 질의 응답 시스템 과정
# 문서들 전처리 : 
#    MAX 3000자 글자수로 분할해서 chunk 생성 - 불용어 제거 -문장별루 분할.
# 임베딩 : 
#    kpf-sbert-v1.1로  문장 평균 임베딩벡터 구함 - es에 문장별루 단락text와 평균벡터 저장.
#-----------------------------------------------------------------------------------------------------

import os
import random
import numpy as np
import pandas as pd
import time
import random

from utils import MyUtils
from utils import get_sentences

myutils = MyUtils(yam_file_path='./data/settings.yaml')
myutils.seed_everything()  # seed 설정
DEVICE = myutils.GPU_info() # GPU 혹은 CPU

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

False
device: cpu


In [2]:
# DATA_FOLDER에 파일들을 불러와서 DF로 만듬.
# -문서는 MAX_LEN 길이만큼씩 잘라서 만듬.

# 파일이 여러개인 경우 폴더 지정
#DATA_FOLDER = './doc/사규개정-out-renew/'
DATA_FOLDER = './doc/company/'
MAX_LEN = 1000  # MAX_LEN 까지 글자수로 구분

files = myutils.getListOfFiles(DATA_FOLDER)
assert len(files) > 0 # files가 0이면 assert 발생
print('*file_count: {}, file_list:{}'.format(len(files), files[0:5]))

# 파일 경로를 지정하면됨.
#files = ["../data11/mpower_doc/신입사원교육.txt",]
docid = 0  # **카운터가 문서에 uid가 되므로, 유일무이한 값므로 지정할것.
count = 0

titles = []
contextids = []
contexts = []

for idx, file_path in enumerate(files):
    if '.ipynb_checkpoints' not in file_path:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = f.read()
            #print(data)
            data_list = [data[i:i+MAX_LEN] for i in range(0, len(data), MAX_LEN)]
            
            docid += 1
            count = 0
            # titles, contextids 임의로 구해서 데이터 프레임 만듬.
            for i, chunk in enumerate(data_list):
                count += 1
                # 파일명만 추출하여 titles에 저장
                #filename = os.path.basename(file_path)
                filename, ext = os.path.splitext(os.path.basename(file_path))  # 파일명과 확장자(.txt, .doc 등) 분리
                titles.append(filename)
                contextids.append(f'{docid}_{count}')
                contexts.append(filename + '\n\n' + chunk.strip()) # 두번째는 문서 제목+문서내용 합처서 문장만듬.
 
# 데이터 프레임으로 만듬.
df_contexts = pd.DataFrame((zip(contexts, titles, contextids)), columns = ['context','question', 'contextid'])     

print(f'*문단 총 계수:{len(df_contexts)}\n')

*file_count: 47, file_list:['./doc/company/16.이사회운영규정_22.01.25.doc.txt', './doc/company/06.대학교 학자금 대출 지원 제도_20.02.01.docx.txt', './doc/company/32.임원상여금지급규정_22.02.07.doc.txt', './doc/company/27.프로젝트 교통비 지원_22.01.25.docx.txt', './doc/company/39.회계업무처리규정_22.01.25.docx.txt']
*문단 총 계수:224



In [3]:
# JSON 파일로 저장해둠.
import json
examples = []

for context, title, contextid in zip(contexts, titles, contextids):
    doc = {}
    doc['context'] = context  # 문단
    doc['title'] = title      # 제목
    doc['contextid'] = contextid # 문서 id
                        
    examples.append(doc)
                        
docs ={}
docs['text'] = examples

#json 파일로 저장.
# JSON 파일을 엽니다.
with open("./data/moco_context.json", "w") as f:
    # 리스트를 JSON으로 변환합니다.
    json.dump(docs, f, indent=4)

In [4]:
# 문장들로 분리
doc_sentences = get_sentences(df=df_contexts, remove_sentence_len=1, remove_duplication=False)

print(f'len:{len(doc_sentences)}, 1: {doc_sentences[1]}')

  0%|          | 0/224 [00:00<?, ?it/s][Kss]: Because there's no supported C++ morpheme analyzer, Kss will take pecab as a backend. :D
For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://github.com/hyunwoongko/python-mecab-kor
- konlpy.tag.Mecab: https://konlpy.org/en/latest/api/konlpy.tag/#mecab-class

100%|██████████| 224/224 [01:45<00:00,  2.13it/s]

*[get_sentences] 문장처리=>len:24, time:105.1116
*[get_sentences] 문장 길이=>평균:28.388392857142858 / MAX: 111 / MIN: 3

len:224, 1: ['이사회운영규정_22.01.25.doc', '의에 출석하지 아니하고 모든 이사가 음성을 동시에 송수신하는 원격통신수단에 의하여 결의에 참가하는 것을 허용할 수 있으며, 이 경우 당해 이사는 이사회에 직접 출석한 것으로 본다.', '③이사회의 결의에 관하여 특별한 이해관계가 있는 이사는 의결권을 행사하지 못한다.', '④제3항의 규정에 의하여 의결권을 행사할 수 없는 이사의 수는 출석한 이사의 수에 산입하지 아니한다.', '제10조(부의사항) ①이사회에 부의할 사항은 다음과 같다.', ' 주주총회에 관한 사항', '1) 주주총회의 소집', '1)2 전자적 방법에 의한 의결권의 행사허용', '2) 영업보고서의 승인', '3) 재무제표의 승인(상법 제449조의2 제1항 단서의 요건을 충족하여 이사회 결의만으로 재무제표 승인이 가능한 경우를 포함한다.)', '4) 정관의 변경', '5) 자본의 감소', '6) 회사의 포괄적 주식교환이전, 해산, 합병, 분할분할합병, 회사의 계속 등', '7) 회사의 영업 전부 또는 중요한 일부의 양도 및 회사의 영업에 중대한 영향을 미치는 다른 회사의 영업 전부 또는 일부의 양수', '8) 영업 전부의 임대 또는 경영위임, 타인과 영업의 손익 전부를 같이하는 계약, 기타 이에 준할 계약의 체결이나 변경 또는 해약', '9) 이사, 감사의 선임 및 해임', '10) 주식의 액면미달발행', '11) 이사의 회사에 대한 책임의 감면', '12) 현금주식현물배당 결정(정관에 의해 상법 제449조의2 제1항 단서의 조건을 충족한 경우 이사회 결의만으로 재무제표 승인이 가능한 경우를 포함한다.)', '13) 주식매수선택권의 부여', '14) 이사의 보수 한도', '15) 회사의 최대주주(그의 특수관계인을 포함함) 및 특수관계

In [5]:
# 인덱스 추가
from tqdm.notebook import tqdm
from utils import embed_text, bi_encoder, mpower_index_batch, create_index, clustering_embedding, kmedoids_clustering_embedding

# ES 관련
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import bulk
    
# 조건에 맞게 임베딩 처리하는 함수 
def embedding(paragraphs:list)->list:
    # 한 문단에 대한 40개 문장 배열들을 한꺼번에 임베딩 처리함
    embeddings = embed_text(model=BI_ENCODER1, paragraphs=paragraphs, return_tensor=False).astype(FLOAT_TYPE)    
    return embeddings

#---------------------------------------------------------------------------
#문단에 문장들의 임베딩을 구하여 각각 클러스터링 처리함.
#---------------------------------------------------------------------------
def index_data(es, df_contexts, doc_sentences:list):
    #클러스터링 계수는 문단의 계수보다는 커야 함. 
    #assert num_clusters <= len(doc_sentences), f"num_clusters:{num_clusters} > len(doc_sentences):{len(doc_sentences)}"
    #-------------------------------------------------------------
    # 각 문단의 문장들에 벡터를 구하고 리스트에 저장해 둠.
    start = time.time()
    cluster_list = []

    rfile_names = df_contexts['contextid'].values.tolist()
    rfile_texts = df_contexts['context'].values.tolist()

    if OUT_DIMENSION == 0:
        dimension = 768
    else:
        dimension = 128

    clustering_num = NUM_CLUSTERS
        
    docs = []
    count = 0
    for i, sentences in enumerate(tqdm(doc_sentences)):
        embeddings = embedding(sentences)
        if i < 3:
            print(f'[{i}] sentences-------------------')
            if len(sentences) > 5:
                print(sentences[:5])
            else:
                print(sentences)
                
            myutils.log_message(f'*[index_data] embeddings.shape: {embeddings.shape}', log_folder='./log/documet')
            print()
        
        #----------------------------------------------------------------
        multiple = 1
        
        # [bong][2023-04-28] 임베딩 출력 계수에 따라 클러스터링 계수를 달리함.
        if NUM_CLUSTERS_VARIABLE == True:
            embeddings_len = embeddings.shape[0]
            if embeddings_len > 2000:
                multiple = 6
            elif embeddings_len > 1000:
                multiple = 5 # 5배
            elif embeddings_len > 600:
                multiple = 4 # 4배
            elif embeddings_len > 300:
                multiple = 3 # 3배
            elif embeddings_len > 100:
                multiple = 2 # 2배
        #----------------------------------------------------------------
        
        # 0=문장클러스터링 임베딩
        if EMBEDDING_METHOD == 0:
            if CLUSTRING_MODE == "kmeans":
                # 각 문단에 분할한 문장들의 임베딩 값을 입력해서 클러스터링 하고 평균값을 구함.
                # [bong][2023-04-28] 문장이 많은 경우에는 클러스터링 계수를 2,3배수로 함
                emb = clustering_embedding(embeddings = embeddings, outmode=OUTMODE, num_clusters=(clustering_num*multiple), seed=SEED).astype(FLOAT_TYPE) 
            else:
                emb = kmedoids_clustering_embedding(embeddings = embeddings, outmode=OUTMODE, num_clusters=(clustering_num*multiple), seed=SEED).astype(FLOAT_TYPE) 
            
        # 1= 문장평균임베딩
        elif EMBEDDING_METHOD == 1:
            # 문장들에 대해 임베딩 값을 구하고 평균 구함.
            arr = np.array(embeddings).astype(FLOAT_TYPE)
            emb = arr.mean(axis=0).reshape(1,-1) #(128,) 배열을 (1,128) 형태로 만들기 위해 reshape 해줌
            clustering_num = 1  # 평균값일때는 NUM_CLUSTERS=1로 해줌.
        # 2=문장임베딩
        else:
            emb = embeddings

        if i < 3:
            myutils.log_message(f'*[index_data] cluster emb.shape: {emb.shape}', log_folder='./log/documet')
            print()
        
        #--------------------------------------------------- 
        # docs에 저장 
        #  [bong][2023-04-28] 여러개 벡터인 경우에는 벡터를 10개씩 분리해서 여러개 docs를 만듬.
        for j in range(multiple):
            count += 1
            doc = {}                                #dict 선언
            doc['rfile_name'] = rfile_names[i]      # contextid 담음
            doc['rfile_text'] = rfile_texts[i]      # text 담음.
            doc['dense_vectors'] = emb[j * clustering_num : (j+1) * clustering_num] # emb 담음.
            docs.append(doc)
        #---------------------------------------------------    

            if count % BATCH_SIZE == 0:
                mpower_index_batch(es, ES_INDEX_NAME, docs, vector_len=clustering_num, dim_size=dimension)
                docs = []
                myutils.log_message("[index_data](1) Indexed {} documents.".format(count), log_folder='./log/documet')
                 

    if docs:
        mpower_index_batch(es, ES_INDEX_NAME, docs, vector_len=clustering_num, dim_size=dimension)
        myutils.log_message("[index_data](2) Indexed {} documents.".format(count), log_folder='./log/documet')

    es.indices.refresh(index=ES_INDEX_NAME)

    myutils.log_message(f'*인덱싱 시간 : {time.time()-start:.4f}\n', log_folder='./log/documet')
    print()
#---------------------------------------------------------------------------


In [7]:
# param--------------------------------------------------------------------
OUT_DIMENSION =128   # 128 혹은 768이면 0입력
EMBEDDING_METHOD=0  # 0=클러스터링 임베딩, 1=문장평균임베딩, 2=문장임베딩
NUM_CLUSTERS=10      # 클러스터링 임베딩일때 클러스터링 수 
NUM_CLUSTERS_VARIABLE=False # 클러스터링 임베딩일때 클러스터링을 문장계수마다 다르계할지.
CLUSTRING_MODE = "kmeans"  # "kmeans" = k-평균 군집 분석, kmedoids =  k-대표값 군집 분석
OUTMODE = "mean"           # kmeans 일때=>mean=평균임베딩값, max=최대임베딩값.,  kmedoids 일때 =>mean=평균임베딩값, medoid=대표임베딩값.

MODEL_PATH = './model/kpf-sbert-128d-v1'#'../../../data11/model/kpf-sbert-v1.1'
POLLING_MODE = 'mean' # 폴링모드 

FLOAT_TYPE = 'float16' # float32 혹은 float16
SEED = 111

# ES 접속
ES_URL = 'http://192.168.0.51:9200/'            # es 접속 주소
ES_INDEX_NAME = 'qaindex_128_10_no'          # 생성혹은 추가할 인덱스 명
ES_INDEX_FILE = './data/mpower10u_128d_10.json'  # 인덱스 구조 파일경로
BATCH_SIZE=20       # ES 배치 사이즈
CREATE_INDEX = True # True이면 기존에 인덱스가 있다면 제거하고 다시 생성.
# param--------------------------------------------------------------------

es = Elasticsearch(ES_URL)
create_index(es, ES_INDEX_FILE, ES_INDEX_NAME, create=CREATE_INDEX)

# 임베딩 모델 로딩
WORD_EMBDDING_MODEL1, BI_ENCODER1 = bi_encoder(model_path=MODEL_PATH, max_seq_len=512, do_lower_case=True, 
                                               pooling_mode=POLLING_MODE, out_dimension=OUT_DIMENSION, device=DEVICE)

print(BI_ENCODER1)
print()
try:
    index_data(es, df_contexts, doc_sentences)
except Exception as e:
    error = f'index_data fail'
    msg = f'{error}=>{e}'
    myutils.log_message(f'/embed/es {msg}', log_folder='./log/documet')

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 128, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)



  0%|          | 0/224 [00:00<?, ?it/s]

[0] sentences-------------------
['이사회운영규정_22.01.25.doc', '이사회 운영규정', '제1장 총 칙', '제1조(목적) 이 규정은 이사회의 효율적인 운영을 위하여 필요한 사항을 규정함을 목적으로 한다.', '제2조(적용범위) 이사회에 관한 사항은 법령 또는 정관에 정하여진 것 이외에는 이 규정이 정하는 바에 의한다.']



/usr/local/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[1] sentences-------------------
['이사회운영규정_22.01.25.doc', '의에 출석하지 아니하고 모든 이사가 음성을 동시에 송수신하는 원격통신수단에 의하여 결의에 참가하는 것을 허용할 수 있으며, 이 경우 당해 이사는 이사회에 직접 출석한 것으로 본다.', '③이사회의 결의에 관하여 특별한 이해관계가 있는 이사는 의결권을 행사하지 못한다.', '④제3항의 규정에 의하여 의결권을 행사할 수 없는 이사의 수는 출석한 이사의 수에 산입하지 아니한다.', '제10조(부의사항) ①이사회에 부의할 사항은 다음과 같다.']




...

/usr/local/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/usr/local/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.9/site-packages/sklearn/cluster/_kmeans.p

  0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/usr/local/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.9/site-packages/sklearn/cluster/_kmeans.p

  0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
